In [1]:
import tkinter as tk
import random

# Set up the window
root = tk.Tk()
root.geometry("800x600")
root.title("Car Racing Game")

#initial parameters
board_width = 800; board_height = 600
road_margin = 100; road_height = board_height - road_margin
car_width = 80; car_height = 30
car_speed = 30; obstacle_speed = 15
#add circle function to tkinter instead of create_oval(x,x,y,y)
def _create_circle(self, x, y, r, **kwargs):
    return self.create_oval(x-r, y-r, x+r, y+r, **kwargs)
tk.Canvas.create_circle = _create_circle
# Create the canvas
canvas = tk.Canvas(root, width=board_width, height=board_height, bg="white")
canvas.pack()

# Create the road
road = canvas.create_rectangle(0, road_margin/2, board_width, road_height+road_margin/2, fill="gray")
canvas.create_line(0,board_height/2, board_width, board_height/2, dash=(5,1))

## Create the car and initial position
#wheels
canvas.create_circle(3*car_width/4, board_height/2+car_height/2, 10, fill="black", tag="car", outline='gray')
canvas.create_circle(5*car_width/4, board_height/2+car_height/2, 10, fill="black", tag="car", outline='gray')
#head light
canvas.create_circle(3*car_width/2, board_height/2, 3, fill="yellow", tag="car", outline='gray')
#body
car=canvas.create_rectangle(car_width/2,
                        board_height/2-car_height/2,
                        3*car_width/2,
                        board_height/2+car_height/2,
                        fill="red", tag="car")
#roof
roof_points=[car_width/2+10, board_height/2-car_height/2,#point 1
             car_width/2+10+10, board_height/2-car_height/2-10,
             3*car_width/2-10-10, board_height/2-car_height/2-10,
             3*car_width/2-10, board_height/2-car_height/2]
canvas.create_polygon(roof_points,fill="yellow", tag="car", outline='magenta')
#anchor point
car_x = car_width/2
car_y = board_height/2


# Create the obstacles, initialize
obstacles = []
for i in range(9):
    x1 = random.randint(board_width+50, board_width+300)
    y1 = random.randint(road_margin/2, road_height)
    x2 = x1 + random.randint(50, car_width)
    y2 = y1 + random.randint(30, car_height)
    obstacle = canvas.create_rectangle(x1, y1, x2, y2, fill="black")
    obstacles.append(obstacle)
    #check overlap

# Move the car left-up / right-down or forward/backward
def move_car(event):
    global car_y
    if event.keysym == "Up" and car_y > road_margin/2+car_height:
        car_y -= car_speed
        canvas.move("car",0,-car_speed)
    elif event.keysym == "Down" and car_y < road_height+road_margin/2-car_height:
        car_y += car_speed
        canvas.move("car",0,car_speed)
    #canvas.coords(car, car_x, car_y, car_x+car_width, car_y+car_height)

# Move the obstacles down the screen
def move_obstacles():
    global obstacles, obstacle_speed
    for obstacle in obstacles:
        canvas.move(obstacle, -obstacle_speed, 0)
        x1, y1, x2, y2 = canvas.coords(obstacle)
        if x1 < 0:
            canvas.move(obstacle, board_width, 0) #move to right
            #reshape the obstacles instead of destroy & recreating
            x1 = random.randint(board_width+50, board_width+600)
            y1 = random.randint(road_margin/2, road_height)
            x2 = x1 + random.randint(50, car_width)
            y2 = y1 + random.randint(30, car_height)
            canvas.coords(obstacle, x1, y1, x2, y2)
    
    # Check for collisions (car vs obstacle)
    car_coords = canvas.coords(car)
    for obstacle in obstacles:
        if canvas.coords(obstacle)[1] < car_coords[3] and canvas.coords(obstacle)[3] > car_coords[1]:
            if canvas.coords(obstacle)[0] < car_coords[2] and canvas.coords(obstacle)[2] > car_coords[0]:
                #show the game over
                canvas.create_text(board_width/2, board_height/2, text="Game Over!", font=("Helvetica", 40), tag="GameOver")
                canvas.create_text(board_width/2, board_height/2+100, fill="green", text="Press space to restart!", font=("Helvetica", 30), tag="restart")

                #disable the car move
                root.unbind("<Key>")
                obstacle_speed=0
                #disable the obstacle move
                break

    # Call the function again after t=50 milliseconds
    root.after(50, move_obstacles)
    
    #check obstacle overlap
def restart(event):
    global obstacles,obstacle_speed
    for obstacle in obstacles:
        canvas.move(obstacle, board_width, 0)
    obstacle_speed=15
    canvas.delete("GameOver","restart")
    root.bind("<Key>", move_car)

# Bind the left and right arrow keys to the move_car function
root.bind("<Key>", move_car)
root.bind("<space>", restart)
# Call the move_obstacles function to start the game
move_obstacles()

# Start the event loop
root.mainloop()